In [2]:
import auxiliary as aux
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

In [3]:
# ------------------------------------------------------------------
# Helper: fitness for ONE mutated sentence  (uses aux.compute_metrics)
# ------------------------------------------------------------------
from copy import deepcopy
import random

from tqdm import tqdm


def fitness_score(model, datum, mutated_ids):
    _, preds, _, _ = aux.inference(
        model,
        torch.tensor([mutated_ids]),
        torch.tensor([datum["attention_mask"]])
    )
    metrics = aux.compute_metrics([preds.tolist()[0]], [datum["labels"]])
    return 1.0 - metrics["recall"]   # maximise mismatch ⟹ minimise recall


# ------------------------------------------------------------------
# Evolutionary attack on ONE sentence
# ------------------------------------------------------------------
def evolve_sentence(
    model,
    datum,
    valid_token_ids,
    pop_size=30,
    n_generations=20,
    mutation_rate=0.15,
    elite_frac=0.2,
    target_recall=0.2,
    seed=42,
):
    random.seed(seed)
    original = datum["input_ids"]
    seq_len  = len(original)

    def random_mutation(base):
        child = base.copy()
        for i, lab in enumerate(datum["labels"]):
            if lab == 18 and random.random() < mutation_rate:
                child[i] = random.choice(valid_token_ids)
        return child

    population = [random_mutation(original) for _ in range(pop_size)]

    for _ in range(n_generations):
        fitness_vals = [fitness_score(model, datum, ind) for ind in population]
        ranked = sorted(zip(population, fitness_vals), key=lambda x: x[1], reverse=True)
        best_ind, best_fit = ranked[0]
        best_rec = 1.0 - best_fit
        if best_rec <= target_recall:
            break

        n_elite = max(1, int(elite_frac * pop_size))
        elites  = [deepcopy(ind) for ind, _ in ranked[:n_elite]]

        offspring = []
        while len(offspring) < pop_size - n_elite:
            parent = random.choice(elites)
            child  = random_mutation(parent)
            if random.random() < 0.3:          # crossover
                other  = random.choice(elites)
                pt     = random.randint(1, seq_len - 2)
                child  = child[:pt] + other[pt:]
            offspring.append(child)

        population = elites + offspring

    return best_ind, best_rec

# ------------------------------------------------------------------
# Attack the whole dataset and report new metrics
# ------------------------------------------------------------------
def adversarial_dataset(
    model,
    tokenizer,
    dataset,
    pop_size=30,
    n_generations=20,
    mutation_rate=0.15,
):
    # Cache normal vocab minus specials
    special_ids = set(tokenizer.all_special_ids)
    valid_token_ids = [tid for tid in tokenizer.get_vocab().values()
                       if tid not in special_ids]

    adv_inputs = []
    adv_tokens = []
    adv_labels = []

    for datum in tqdm(dataset, desc="Evolving sentences"):
        best_ids, _ = evolve_sentence(
            model, datum, valid_token_ids,
            pop_size=pop_size,
            n_generations=n_generations,
            mutation_rate=mutation_rate,
        )
        adv_inputs.append(best_ids)
        adv_tokens.append(tokenizer.convert_ids_to_tokens(best_ids))
        adv_labels.append(datum["labels"])   # labels unchanged

    # Evaluate the whole adversarial corpus
    preds = []
    for ids, datum in tqdm(zip(adv_inputs, dataset),
                           total=len(dataset),
                           desc="Inference on adversarial set"):
        _, p, _, _ = aux.inference(
            model,
            torch.tensor([ids]),
            torch.tensor([datum["attention_mask"]])
        )
        preds.append(p.tolist()[0])

    dataset = dataset.add_column('adv_inputs', adv_inputs)
    dataset = dataset.add_column('adv_tokens', adv_tokens)

    metrics = aux.compute_metrics(preds, adv_labels)
    return metrics, preds, adv_inputs, dataset

In [4]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# Suppress only the specific warning from seqeval
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# 1️⃣ load everything exactly as you did
model      = AutoModelForTokenClassification.from_pretrained('to_share/distilbert1')
tokenizer  = AutoTokenizer.from_pretrained('to_share/distilbert1')
train_ds   = aux.json_to_Dataset("data/distilbert_train.json")

# 2️⃣ run the adversarial evolution
adv_metrics, adv_preds, adv_inputs, dataset = adversarial_dataset(
    model,
    tokenizer,
    train_ds.select(range(10)),    # start small for speed
    pop_size        = 20,
    n_generations   = 10,
    mutation_rate   = 0.05,
)

print("Metrics on evolved examples:")
for k, v in adv_metrics.items():
    if k != "confusion_matrix":
        print(f"{k:>12}: {v:.4f}")


/home/andreafabbricatore/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_f

Metrics on evolved examples:
   precision: 0.4194
      recall: 0.5098
          f1: 0.4602
    accuracy: 0.9360


In [5]:
aux.write_dataset_to_json(dataset,"test.json")